In [3]:
import os
import pandas as pd
import numpy as np
import csv

In [4]:
# options
# pd.options.display.max_columns = None # show all columns

In [5]:
# paths to data
data_dir = os.getcwd() + "/data/"
X_train_path = data_dir + "X_train.csv"
census_path = data_dir + "ACS_17_5YR_DP05_with_ann.csv"
socioeconomic_path = data_dir + "SocioEconomic_2017.csv"

# Load the ATD Data
We have to do this first or the kernel crashes with an OOM error

In [6]:
X_train = pd.read_csv(X_train_path, index_col=0)

c:\users\patty\appdata\local\programs\python\python37-32\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### Collect all the distribution center zipcodes

In [7]:
zipcodes = X_train["DC_ZIPCODE"].unique()
zipcodes

array([11717, 12206, 14624, 15026, 20110, 21220, 21227, 22802, 24019,
       24382, 25159, 27215, 27893, 28070, 28214, 28311, 28403, 28704,
       29201, 29406, 29501, 29662, 30084, 30294, 30909, 31008, 32254,
       32303, 32514, 32824, 33025, 33619, 33916, 35211, 35810, 36117,
       36582, 37416, 37686, 37912, 38141, 39218, 43035, 44146, 44706,
       45840, 46241, 48375, 53154, 55416, 57104, 58078, 63044, 64161,
       65803, 67217, 68521, 70460, 70817, 71854, 72114, 73179, 75006,
       77067, 77489, 77705, 78501, 78753, 79118, 79382, 79928, 80216,
       80916, 81505, 83687, 85007, 87113, 89115, 89434, 90221, 90670,
       91730, 91911, 93021, 93263, 93725, 94801, 95131, 95838, 97203,
       97502, 98001], dtype=int64)

# Load the Population Data

### Import 2017 census information from CSV

In [8]:
census = pd.read_csv(census_path, skiprows=[0])

c:\users\patty\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4,8,12,20,32,36,40,44,48,60,76,84,96,100,104,108,116,120,124,132,252,280,284,304) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
census.head()

,Id,Id2,Geography,Estimate; SEX AND AGE - Total population,Margin of Error; SEX AND AGE - Total population,Percent; SEX AND AGE - Total population,Percent Margin of Error; SEX AND AGE - Total population,Estimate; SEX AND AGE - Total population - Male,Margin of Error; SEX AND AGE - Total population - Male,Percent; SEX AND AGE - Total population - Male,...,"Percent; CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population","Percent Margin of Error; CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population","Estimate; CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population - Male","Margin of Error; CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population - Male","Percent; CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population - Male","Percent Margin of Error; CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population - Male","Estimate; CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population - Female","Margin of Error; CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population - Female","Percent; CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population - Female","Percent Margin of Error; CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population - Female"
0,8600000US00601,601,ZCTA5 00601,17599,261,17599,(X),8809,167,50.1,...,13511,(X),6464,142,47.8,0.5,7047,124,52.2,0.5
1,8600000US00602,602,ZCTA5 00602,39209,128,39209,(X),19231,58,49.0,...,31093,(X),15146,84,48.7,0.3,15947,163,51.3,0.3
2,8600000US00603,603,ZCTA5 00603,50135,805,50135,(X),24497,463,48.9,...,39243,(X),18781,371,47.9,0.5,20462,362,52.1,0.5
3,8600000US00606,606,ZCTA5 00606,6304,255,6304,(X),3148,163,49.9,...,4896,(X),2362,150,48.2,1.6,2534,129,51.8,1.6
4,8600000US00610,610,ZCTA5 00610,27590,163,27590,(X),13332,74,48.3,...,21884,(X),10364,87,47.4,0.3,11520,131,52.6,0.3


### We only care about the zip code and total population

In [10]:
population = census[["Id2", "Estimate; SEX AND AGE - Total population"]]
population.columns = ["Zipcode", "Population"]

In [11]:
population.head()

,Zipcode,Population
0,601,17599
1,602,39209
2,603,50135
3,606,6304
4,610,27590


### We only care about the populations at the distribution centers

In [12]:
atd_populations = population[population["Zipcode"].isin(zipcodes)]
atd_populations.head()

,Zipcode,Population
2972,11717,63207
3222,12206,16036
4208,14624,36265
4370,15026,3661
6246,20110,48312


### Augment the data

In [19]:
zipdict = atd_populations.groupby("Zipcode").sum().to_dict()["Population"]
X_train["DC_ZIPCODE_POPULATION"] = X_train["DC_ZIPCODE"].map(lambda zipcode: zipdict.get(zipcode))

In [23]:
X_train.head(20000)["DC_ZIPCODE_POPULATION"].describe()

count    20000.0
mean     63207.0
std          0.0
min      63207.0
25%      63207.0
50%      63207.0
75%      63207.0
max      63207.0
Name: DC_ZIPCODE_POPULATION, dtype: float64

In [18]:
X_train.head(20000).join(atd_populations.set_index('Zipcode'), on='DC_ZIPCODE')

,idx,DC_ZIPCODE,Invoice_Year,Invoice_Week,CATEGORY,TIER,SPEED_RATING_CODE,RIM_DIAMETER_SIZE_CODE,WIDTH,HEIGHT,...,Monthly_Top_2_Customer_Total_Sales,Monthly_Top_3_Customer_Total_Sales,Monthly_Top_4_Customer_Total_Sales,Monthly_Top_5_Customer_Total_Sales,Monthly_Top_6_Customer_Total_Sales,Monthly_Top_7_Customer_Total_Sales,Monthly_Top_8_Customer_Total_Sales,Monthly_Top_9_Customer_Total_Sales,Monthly_Top_10_Customer_Total_Sales,Population
0,0,11717,2016,44,Passenger Car / Mini-Van,Tier 3,S,13.0,6.89,22.680,...,1026564.0,427710.0,574404.0,690830.0,258324.0,235820.0,160756.0,183788.0,114146.0,63207
1,1,11717,2017,10,Passenger Car / Mini-Van,Tier 3,S,13.0,6.89,22.680,...,1087224.0,157358.0,342798.0,501052.0,252610.0,107190.0,272140.0,132466.0,127240.0,63207
2,2,11717,2017,13,Passenger Car / Mini-Van,Tier 3,S,13.0,6.89,22.680,...,1087224.0,157358.0,342798.0,501052.0,252610.0,107190.0,272140.0,132466.0,127240.0,63207
3,3,11717,2017,19,Passenger Car / Mini-Van,Tier 3,H,13.0,6.89,22.640,...,314464.0,295766.0,386680.0,217224.0,187662.0,117520.0,50656.0,55520.0,86772.0,63207
4,4,11717,2017,25,Passenger Car / Mini-Van,Tier 3,S,13.0,6.89,22.680,...,260474.0,396232.0,259164.0,378726.0,525926.0,246766.0,155972.0,83740.0,73248.0,63207
5,5,11717,2016,29,Passenger Car / Mini-Van,Tier 2,T,13.0,6.97,22.680,...,436070.0,322108.0,472728.0,214580.0,179086.0,135796.0,226266.0,128360.0,143966.0,63207
6,6,11717,2016,30,Passenger Car / Mini-Van,Tier 2,T,13.0,6.97,22.680,...,436070.0,322108.0,472728.0,214580.0,179086.0,135796.0,226266.0,128360.0,143966.0,63207
7,7,11717,2016,36,Passenger Car / Mini-Van,Tier 2,T,13.0,6.97,22.680,...,458416.0,607214.0,284448.0,562312.0,224766.0,199210.0,172620.0,48782.0,107648.0,63207
8,8,11717,2016,41,Passenger Car / Mini-Van,Tier 2,T,13.0,6.97,22.680,...,1026564.0,427710.0,574404.0,690830.0,258324.0,235820.0,160756.0,183788.0,114146.0,63207
9,9,11717,2017,8,Passenger Car / Mini-Van,Tier 2,T,13.0,6.97,22.680,...,416070.0,351978.0,390446.0,307034.0,144974.0,187884.0,620762.0,100636.0,42290.0,63207


In [12]:
X_train.head()

,idx,DC_ZIPCODE,Invoice_Year,Invoice_Week,CATEGORY,TIER,SPEED_RATING_CODE,RIM_DIAMETER_SIZE_CODE,WIDTH,HEIGHT,...,Monthly_Top_2_Customer_Total_Sales,Monthly_Top_3_Customer_Total_Sales,Monthly_Top_4_Customer_Total_Sales,Monthly_Top_5_Customer_Total_Sales,Monthly_Top_6_Customer_Total_Sales,Monthly_Top_7_Customer_Total_Sales,Monthly_Top_8_Customer_Total_Sales,Monthly_Top_9_Customer_Total_Sales,Monthly_Top_10_Customer_Total_Sales,DC_ZIPCODE_POPULATION
0,0,11717,2016,44,Passenger Car / Mini-Van,Tier 3,S,13.0,6.89,22.68,...,1026564.0,427710.0,574404.0,690830.0,258324.0,235820.0,160756.0,183788.0,114146.0,63207.0
1,1,11717,2017,10,Passenger Car / Mini-Van,Tier 3,S,13.0,6.89,22.68,...,1087224.0,157358.0,342798.0,501052.0,252610.0,107190.0,272140.0,132466.0,127240.0,63207.0
2,2,11717,2017,13,Passenger Car / Mini-Van,Tier 3,S,13.0,6.89,22.68,...,1087224.0,157358.0,342798.0,501052.0,252610.0,107190.0,272140.0,132466.0,127240.0,63207.0
3,3,11717,2017,19,Passenger Car / Mini-Van,Tier 3,H,13.0,6.89,22.64,...,314464.0,295766.0,386680.0,217224.0,187662.0,117520.0,50656.0,55520.0,86772.0,63207.0
4,4,11717,2017,25,Passenger Car / Mini-Van,Tier 3,S,13.0,6.89,22.68,...,260474.0,396232.0,259164.0,378726.0,525926.0,246766.0,155972.0,83740.0,73248.0,63207.0


In [13]:
X_train.to_csv('train_augmented_1.csv')

# Load the Socioeconomic Data

### Import socioeconomic data from CSV

In [14]:
incomes = pd.read_csv(socioeconomic_path, skiprows=[0])

We have already filtered the columns we want, so we only want to rename the headers

In [15]:
incomes.columns = ["zipcode", "population_employed", "population_commuting", "total_household_income", "median_household_income", "mean_household_income"]
incomes.head(10)

,zipcode,population_employed,population_commuting,total_household_income,median_household_income,mean_household_income
0,601,14031,3477,5818,11757,18970
1,602,32412,11714,12719,16190,24179
2,603,41064,12021,19009,16645,27627
3,606,5179,1503,1959,13387,16739
4,610,22797,8414,9120,18741,25085
5,612,51267,15231,23037,17744,25454
6,616,9257,2796,3606,14918,23073
7,617,19688,5468,8079,17157,22839
8,622,6361,1525,2330,16727,32910
9,623,34603,10956,14165,16401,24124


### We only care about the socioeconomic status at the distribution centers

In [16]:
atd_incomes = incomes[incomes["zipcode"].isin(zipcodes)]
atd_incomes.head()

,zipcode,population_employed,population_commuting,total_household_income,median_household_income,mean_household_income
2972,11717,48504,31180,13587,74202,93757
3222,12206,11809,6532,6147,31812,48686
4208,14624,30235,19420,13979,64645,76730
4370,15026,2900,1775,1345,70243,81002
6246,20110,37289,23980,14098,79798,95608


### Augment the data

In [32]:
out_file = "data/train_augmented_2.csv"
open(out_file, 'w').close()
with open(out_file, 'a') as f:
    writeHeader = True
    for chunk in pd.read_csv("data/train_augmented_1.csv", index_col=0, chunksize=10 ** 5):
        aug = chunk.join(atd_incomes.set_index('zipcode'), on='DC_ZIPCODE')
        aug.to_csv(f, header=writeHeader)
        writeHeader = False